This notebook shows how to access the truth trajectory information for each event.

This information is extracted from the simulation and is available to help quantify the accuracy and precision of our reconstruction and analysis algorithms.

We also show how to project that information into the wireplane images. This provides and overview of the different "times" that we deal with in the experiment.

In [ ]:
import os,sys
import plotly as pl
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

In [ ]:
# on trex, for some reason, need to load ROOT in separate cell before loading icdl modules
import ROOT as rt

In [ ]:
from larlite import larlite
from larcv import larcv
from ublarcvapp import ublarcvapp
from larlite import larutil
import lardly

Below we specify the location of the file we want to use.

For a list of example files, refer to this [list](https://github.com/NuTufts/icdl/wiki/Locations-of-Tutorial-Files).

In [ ]:
# Specify location of the file we want to open

# For list of tutorial files check out []
inputfile = "/tutorial_files/merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"

In [ ]:
# Use ROOT to dump the data of the file
# you'll see a lot of TTree objects. 
# These are essentially tables of data with a specific schema, 
#   with each row of data consisting of one "event".

# For this tutorial we'll need the following TTree(s)
# * mctrack_mcreco_tree
# * mcshower_mcreco_tree

rfile = rt.TFile(inputfile,"open")
rfile.ls()

We do not access the data (i.e. Trees) directly). The ROOT trees store data in the form of serialized instances of custom C++ classes! In otherwords, for each event, copies of C++ classes are turned into a binary string, storing the values of its data members. Unpacking this involves de-serializing the class data and giving back a set of C++ class instances for each event. We have special classes of our own that interfaces with ROOT's IO functions to help us do this.

There is two we need to use.

The first is `larlite::storage_manager` which interfaces data products from larlite, which is a clone of the data products from larsoft. (Why do we use larlite and not larsoft directly? larlite does not require the large number of dependencies that are hard to build on systems that are not on Fermilab.)

The second is `larcv::IOManager` which provides the interface to the images, which we'll use at the end of the notebook.

In [ ]:
# We use the larcv IOManager class to load the trees for us.

# Initialize an storage_manager instance that will read out input file
ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
ioll.add_in_filename( inputfile )
ioll.open()

# Initialize an IOManager instance that will read out input file
iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickBackward )
iolcv.add_in_file( inputfile )
iolcv.reverse_all_products()
iolcv.initialize()



In [ ]:
# Now we load an event

ENTRY_NUM = 0
ioll.go_to(ENTRY_NUM)
iolcv.read_entry(ENTRY_NUM)


In [ ]:
# Get the containers holding the mctrack and mcshower objects

event_mctrack  = ioll.get_data( larlite.data.kMCTrack,  "mcreco" )
event_mcshower = ioll.get_data( larlite.data.kMCShower, "mcreco" )

print("Number of mctracks: ",event_mctrack.size())
print("Number of mcshowers: ",event_mcshower.size())

In [ ]:
# Because we'll need to orient the information within the geometry of the MicroBooNE detector,
# we need to activate it.
larutil.LArUtilConfig.SetDetector( larlite.geo.kMicroBooNE )


In [ ]:
# Before we vizualize this information, we can dump out this information
# We can use a utility that will try to arrange the list of tracks and showers
# by their mother-daughter relationship

# warning, its a bit of a mess

mcpg = ublarcvapp.mctools.MCPixelPGraph()
mcpg.buildgraphonly( ioll )
mcpg.printGraph(0, False)

In [ ]:
import lardly
from lardly.detectoroutline import get_tpc_boundary_plot

tpclines = get_tpc_boundary_plot()


In [ ]:
## Run this block if you want to draw just the TPC plot that we retrieved

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=[tpclines], layout=plot_layout )
fig.show()

Below we'll plot the Truth track trajectories. 

In the lardly code, the following color scheme is used:

```
default_pid_colors = {2212:'rgb(153,55,255)', # protons                                       
                      13:'rgb(255,0,0)', # muons                               
                      -13:'rgb(255,0,0)', # muons                          
                      211:'rgb(255,128,255)',# pions                       
                      -211:'rgb(255,128,255)',# pions                      
                      0:'rgb(0,0,0)'# other                                  
                      }
```

What you should notice is that all the tracks are inside the TPC.
This is because only true trajectory information is saved inside the TPC
(exception is the creation origin) which is kept somewhere in the mctrack object.

The code that takes a container of mctrack objects and returns respective plot objects is

```
lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=False )
```

Note that `apply_t0_offset=False`.

In [ ]:
# make plot objects for the track objects

plot_mctracks = lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=False )



axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=[tpclines]+plot_mctracks, layout=plot_layout )
fig.show()


Now we remake the track objects, but set `apply_t0_offset=False`.

What you'll see is that the tracks are now outside the TPC.
This has to do with the way we measure and reconstruct
the time of trajectories in the detector.


In [ ]:
# make plot objects for the track objects

plot_mctracks = lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=True )


axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=[tpclines]+plot_mctracks, layout=plot_layout )
fig.show()


# Timing in the TPC

To understand the timing in the detector, we need to describe how the data is saved.

There are electronics that continually stores a rolling queue of voltage versus time measurements.

The time between samples (sometimes called ticks) is determined by the electronics clock used. For MicroBooNE, the data acquisition system  (or DAQ) used to record the voltage versus time on all the wires saves a sample every 0.5 microseconds, i.e. 0.5 microseconds per tick.



In [ ]:
# get detector outlines

PLANE = 0 # options are 0,1, or 2 for the U,V,Y plane
plane_image = adc_v.at(PLANE)
plane_plot = lardly.data.visualize_larcv_image2d( plane_image, reverse_ticks=True )

# plotly figure
fig = go.Figure( data=[plane_plot] )
fig.show()

## If opening the tutorial file, merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root,
## the image should be of a Charged-Current Electron Neutrino Interaction
## If on the U-plane, the neutrino interaction vertex is around (1650,5328)

For more information on the classes, please follow the links below.

Used in this tutorial:

* [larcv::IOManager](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/IOManager.h)
* [larcv::Image2D](https://github.com/LArbys/LArCV/blob/develop/larcv/core/DataFormat/Image2D
.h)
* [plotly heatmap](https://plotly.com/python/heatmaps/)